In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


from scipy.stats import skew
from scipy.stats import kurtosistest
import researchpy as rp
import scipy.stats as stats
from scipy.stats import levene
from scipy.stats import kstest
from sklearn.preprocessing import RobustScaler
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [2]:
df = pd.read_csv("listings_limpio_2.csv", index_col = 0)
df.head(2)

,neighbourhood_group,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm
0,Chamartín,Private room,3.021213,1,81,0.57,1,149,3
1,Latina,Private room,2.661819,4,33,0.38,2,131,0


# Ejercicio 1. Estandarizar las columnas de las variables predictoras

Nuestras variables predictoras  numéricas son (v. independientes): `minimum_nights`,`number_of_reviews`,`reviews_per_month`,`calculated_host_listings_count`,`availability_365` `number_of_reviews_ltm`. Otra v. predictora categórica es: `neighbourhood_group`.

In [6]:
df_num = df.select_dtypes(exclude= "object")
df_num.head(2)

,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm
0,3.021213,1,81,0.57,1,149,3
1,2.661819,4,33,0.38,2,131,0


In [22]:
robust = RobustScaler()

robust.fit(df_num)


RobustScaler()

In [23]:
X_robust = robust.transform(df_num)


In [24]:
df_num_robust = pd.DataFrame(X_robust, columns = df_num.columns)
df_num_robust.head(2)


,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm
0,-0.164317,-0.5,2.000000,-0.675766,-0.166667,0.146104,0.285714
1,-0.823490,1.0,0.702703,-0.840983,0.000000,0.087662,-0.142857


Podemos ver que nuestras variables han cambiado.

# Ejercicio 2. Cread un ANOVA, siendo la variable respuesta price y las variables predictoras el resto de las columnas del dataframe

In [13]:
df.columns

Index(['neighbourhood_group', 'room_type', 'price', 'minimum_nights',
       'number_of_reviews', 'reviews_per_month',
       'calculated_host_listings_count', 'availability_365',
       'number_of_reviews_ltm'],
      dtype='object')

In [31]:
columnas = df_num_robust.columns
df[columnas] = df_num_robust
df.head()


,neighbourhood_group,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm
0,Chamartín,Private room,-0.164317,-0.5,2.000000,-0.675766,-0.166667,0.146104,0.285714
1,Latina,Private room,-0.823490,1.0,0.702703,-0.840983,0.000000,0.087662,-0.142857
2,Arganzuela,Entire home/apt,-0.327611,6.5,-0.189189,0.000000,0.666667,0.672078,-0.142857
3,Centro,Entire home/apt,0.253787,1.5,0.189189,-1.032288,-0.166667,0.204545,0.428571
4,Arganzuela,Private room,-0.895642,0.0,3.972973,-0.223592,-0.166667,0.779221,0.571429


In [32]:
lm = ols('price ~ + neighbourhood_group + room_type + minimum_nights + number_of_reviews + reviews_per_month + calculated_host_listings_count + availability_365 + number_of_reviews_ltm', data=df).fit()
sm.stats.anova_lm(lm)


,df,sum_sq,mean_sq,F,PR(>F)
neighbourhood_group,20.0,39.436250,1.971813,3.412390,3.643034e-07
room_type,3.0,0.763058,0.254353,0.440179,7.242652e-01
minimum_nights,1.0,37.206350,37.206350,64.388776,1.086849e-15
number_of_reviews,1.0,0.470405,0.470405,0.814076,3.669306e-01
reviews_per_month,1.0,6.116179,6.116179,10.584571,1.142559e-03
calculated_host_listings_count,1.0,196.789809,196.789809,340.561616,2.577850e-75
availability_365,1.0,404.927124,404.927124,700.761065,2.210759e-151
number_of_reviews_ltm,1.0,0.179850,0.179850,0.311245,5.769244e-01
Residual,17156.0,9913.407135,0.577839,NaN,NaN


In [33]:
lm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.065
Model:                            OLS   Adj. R-squared:                  0.063
Method:                 Least Squares   F-statistic:                     40.93
Date:                Mon, 30 May 2022   Prob (F-statistic):          2.98e-223
Time:                        10:33:48   Log-Likelihood:                -19658.
No. Observations:               17186   AIC:                         3.938e+04
Df Residuals:                   17156   BIC:                         3.961e+04
Df Model:                          29                                         
Covariance Type:            nonrobust                                         
================================================================================================================
                                                   coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------------
Intercept                                       -0.0639      0.026     -2.477      0.013      -0.115      -0.013
neighbourhood_group[T.Barajas]                   0.0643      0.070      0.913      0.361      -0.074       0.203
neighbourhood_group[T.Carabanchel]              -0.0538      0.040     -1.351      0.177      -0.132       0.024
neighbourhood_group[T.Centro]                   -0.0800      0.026     -3.045      0.002      -0.132      -0.029
neighbourhood_group[T.Chamartín]                -0.0530      0.042     -1.254      0.210      -0.136       0.030
neighbourhood_group[T.Chamberí]                 -0.0393      0.034     -1.144      0.253      -0.107       0.028
neighbourhood_group[T.Ciudad Lineal]            -0.0026      0.042     -0.063      0.950      -0.085       0.079
neighbourhood_group[T.Fuencarral - El Pardo]    -0.1265      0.053     -2.396      0.017      -0.230      -0.023
neighbourhood_group[T.Hortaleza]                -0.0082      0.048     -0.172      0.864      -0.102       0.086
neighbourhood_group[T.Latina]                   -0.0489      0.043     -1.141      0.254      -0.133       0.035
neighbourhood_group[T.Moncloa - Aravaca]        -0.0165      0.042     -0.390      0.697      -0.100       0.067
neighbourhood_group[T.Moratalaz]                 0.1168      0.074      1.571      0.116      -0.029       0.263
neighbourhood_group[T.Puente de Vallecas]        0.0243      0.043      0.566      0.572      -0.060       0.109
neighbourhood_group[T.Retiro]                   -0.0925      0.040     -2.324      0.020      -0.170      -0.014
neighbourhood_group[T.Salamanca]                -0.0156      0.033     -0.470      0.639      -0.081       0.050
neighbourhood_group[T.San Blas - Canillejas]     0.1138      0.045      2.521      0.012       0.025       0.202
neighbourhood_group[T.Tetuán]                   -0.0157      0.037     -0.419      0.675      -0.089       0.058
neighbourhood_group[T.Usera]                     0.0389      0.054      0.714      0.475      -0.068       0.146
neighbourhood_group[T.Vicálvaro]                -0.0803      0.096     -0.834      0.404      -0.269       0.108
neighbourhood_group[T.Villa de Vallecas]        -0.0684      0.090     -0.759      0.448      -0.245       0.108
neighbourhood_group[T.Villaverde]               -0.1756      0.065     -2.710      0.007      -0.303      -0.049
room_type[T.Hotel room]                          0.0685      0.064      1.076      0.282      -0.056       0.193
room_type[T.Private room]                        0.0129      0.013      1.020      0.308      -0.012       0.038
room_type[T.Shared room]                        -0.0275      0.051     -0.542      0.587      -0.127       0.072
minimum_nights                  

El objetivo principal del ANOVA es ver qué variables tienenen más peso a la hora de predecir nuestra variable dependiente, en este caso el precio.

Las hipótesis de una ANOVA son las siguientes: 
   
        - Ho : las variables NO pueden predecir la var. respuesta (precio)

        - H1 : las variables SI pueden predecir el precio.
Con estas hipótesis aceptamos o descartamos las variables que son relevantes para predecir nuestro peso. 

Entonces con nuestros resultados podemos afirmar que:

        - Las variables `minimum_nights`, `calculated_host_listings_count`,`availability_365`, `neighbourhood_group[T.Centro]`, `neighbourhood_group[T.Fuencarral - El Pardo]` `neighbourhood_group[T.Retiro]`,`neighbourhood_group[T.San Blas - Canillejas]`,`neighbourhood_group[T.Villaverde]` definen el valor de la variable precio porque su p-valor es menor a 0.05.

        Esto quiere decir que el precio de la vivienda se ve afectado por el barrio en el que se encuentren (solo en algunos barrios.)
        - Por el contrario, las variables `number_of_reviews` `reviews_per_month` `number_of_reviews_ltm` tienen un p-valor mayor de 0.05, por lo que no podemos rechazar las H0, así que esas variables no pueden predecir nuestra var. respuesta `precio`. 

Estadísticamente, nuestras hipótesis son las siguientes:
Ho : las medias de los diferentes grupos son iguales
h1 : las medias difieren entre sí.